In [0]:
import json

twitter_cred = dict()
'''
twitter_cred['CONSUMER_KEY'] = '***'
twitter_cred['CONSUMER_SECRET'] = '****'
twitter_cred['ACCESS_KEY'] = '***-***'
twitter_cred['ACCESS_SECRET'] = '****'
'''
twitter_cred['CONSUMER_KEY'] = 'pDHudtJoB0opb2U9JoZr6ge2Y'
twitter_cred['CONSUMER_SECRET'] = 'Z39VH8rx45WAlGjSFuuonmjZclo9kWsHPPJs46IobujPbQr5WW'
twitter_cred['ACCESS_KEY'] = '151774740-ahQRcASlsrfDOFAhIYoSb9Kc0q7mWWR3RJ1o1Czq'
twitter_cred['ACCESS_SECRET'] = 'koLzEjvqSZMOfVxEveIZ3VhSHmnGPFaPzAcOU8bzwuhnY'

with open('twitter_credentials.json', 'w') as secret_info:
    json.dump(twitter_cred, secret_info, indent=4, sort_keys=True)

In [0]:
import tweepy
import csv
import json
import pandas as pd
import datetime
import re,string

with open('twitter_credentials.json') as cred_data:
  info = json.load(cred_data)
  consumer_key = info['CONSUMER_KEY']
  consumer_secret = info['CONSUMER_SECRET']
  access_key = info['ACCESS_KEY']
  access_secret = info['ACCESS_SECRET']
  
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)
  
  
all_the_tweets = []
final_op = []
#keywords = ["product","launch","sales","release","revenue","profit","loss","expectation","market","growth","country","satisfaction","satisfied","happy","great","superb","worst","bad","good","unacceptable","apology","sorry"]
account_list = ['@SunflowerLab','@SyberryCorp','@iflexion','@indianic','@BelatrixSF','@zcocorporation','@algoworks','@EchoLogistics','@Softura','@DogtownMedia','@SumatoSoft','@openxcell','@openGeekslab','@DockYard','@promatics','@ZealouSys','@xtreemsolution','@jetrubyagency','@corelley_','@omsoftware','@konstantinfo','@tablexi','@SimbirSoft_Ltd','@tkxel','@endivesoftware','@consagous','@nexsoftsys','@octalitsolution','@Terasol_app','@KitelyTech','@Clavax','@iqlance','@HQSoftware','@mobcoderllc','@brain_mobi']
#account_list = ['@indianic']

startDate = datetime.datetime(2018, 9, 18, 0, 0, 0)
endDate =   datetime.datetime(2019, 9, 17, 0, 0, 0)


if len(account_list) > 0:  
  for target in account_list:
    #print("Getting data for " + target)
    #item = auth_api.get_user(target)
    new_tweets = api.user_timeline(screen_name=target, count=23, include_rts = False, tweet_mode = 'extended', exclude_replies=True)  
    # saving the most recent tweets
    all_the_tweets.extend(new_tweets)
    
    '''for tweet in all_the_tweets:
      outtweets=[]
      #print(tweet)
      if any(keyword in str(tweet.full_text.lower()) for keyword in keywords):
        outtweets = [tweet.id_str, tweet.user.name, tweet.created_at, str(tweet.full_text)]
        final_op.append(outtweets)
        #print(outtweets)  
     '''   
    
    for tweet in all_the_tweets:
      outtweets=[]
      if tweet.created_at < endDate and tweet.created_at > startDate:
        outtweets = [tweet.id_str, tweet.user.name, tweet.created_at,tweet.user.location, str(tweet.full_text),tweet.user.description]
        final_op.append(outtweets)
        #print(tweet)  
        #print(outtweets)
        
        
    '''  
    for tweet in all_the_tweets:
      outtweets=[]
      outtweets = [tweet.id_str, tweet.user.name, tweet.created_at,tweet.user.location, str(tweet.full_text),tweet.user.description]
      final_op.append(outtweets)
      #print(tweet)  
      print(outtweets)  
    '''
#print(final_op)          
df = pd.DataFrame(final_op)
#df = df.transpose()
df.columns = ["Id", "SME", "Tweet_Date","Location", "Tweets","Company_Description"] 
df.drop_duplicates(subset ="Id",inplace=True)          
#df.head()        


def text_separated(dataframe, columnname, keyword):
    twt_txts = []
    for twts in dataframe[columnname]: 
        twt_txts.append(twts)
        
    twts_cleaned = []
    for twts_2 in twt_txts:
        cleaned_twt = twts_2[:twts_2.find(keyword)-1] 
        twts_cleaned.append(cleaned_twt)
    return(twts_cleaned)
  
  
def strip_all_entities(text, removecharacter):
    twt_txts = []
    for twts in text: 
        twt_txts.append(twts)
    
    twts_cleaned = []
    for word in twt_txts:
      #print(word)
      #cleaned_twt = word[:twts_2.find(keyword)-1] 
      cleaned_twt = word.replace(removecharacter, " ")
      #print(cleaned_twt)
      twts_cleaned.append(cleaned_twt)
    return(twts_cleaned)
  
def deEmojify(inputString):
  twt_txts = []
  for twts in inputString: 
    twt_txts.append(twts)
    twts_cleaned = []
  for twts_2 in twt_txts:
    cleaned_twt = twts_2.encode('ascii', 'ignore').decode('ascii')
    twts_cleaned.append(cleaned_twt)
  return twts_cleaned
  
df['Cleaned_Tweets'] = text_separated(df, 'Tweets', 'https')
df['Cleaned_Tweets'] = strip_all_entities(df['Cleaned_Tweets'], "#")
df['Cleaned_Tweets'] = strip_all_entities(df['Cleaned_Tweets'], "@")
df['Cleaned_Tweets'] = deEmojify(df['Cleaned_Tweets'])
df['Cleaned_Tweets'] = strip_all_entities(df['Cleaned_Tweets'], "\n")

export_excel = df.to_excel (r'sme_usa_tweets.xlsx', index = None, header=True)

df.drop(["Id", "Location", "Tweets","Company_Description"], axis = 1, inplace = True) 
export_excel = df.to_excel (r'sme_cleaned_tweets.xlsx', index = None, header=True)


#df['Cleaned_Tweets']
#export_excel = df.to_excel (r'sme_usa_tweets.xlsx', index = None, header=True)

